In [1]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [2]:
# create/connect to db
engine = create_engine('sqlite:///ideas.db', echo=True)

# declare the base
Base = declarative_base()

# define the idea table model
class Idea(Base):
    __tablename__ = 'ideas'

    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    notes = Column(String)
    impact = Column(Integer)
    confidence = Column(Integer)
    ease = Column(Integer)

    @property
    def ice_score(self):
        return (self.impact or 0) * (self.confidence or 0) * (self.ease or 0)


C:\Users\ashvi\AppData\Local\Temp\ipykernel_21264\1540353415.py:5: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Create table if needed
Base.metadata.create_all(engine)

# Set up sessionmaker
Session = sessionmaker(bind=engine)


2023-12-19 16:42:58,166 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-19 16:42:58,167 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ideas")
2023-12-19 16:42:58,168 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-19 16:42:58,169 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ideas")
2023-12-19 16:42:58,170 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-19 16:42:58,173 INFO sqlalchemy.engine.Engine 
CREATE TABLE ideas (
	id INTEGER NOT NULL, 
	title VARCHAR NOT NULL, 
	notes VARCHAR, 
	impact INTEGER, 
	confidence INTEGER, 
	ease INTEGER, 
	PRIMARY KEY (id)
)


2023-12-19 16:42:58,173 INFO sqlalchemy.engine.Engine [no key 0.00081s] ()
2023-12-19 16:42:58,191 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
# CRUD functions

def add_idea(title: str, notes: str, impact: int, confidence: int, ease: int) -> Idea:
    session = Session()
    new_idea = Idea(title=title, notes=notes, impact=impact, confidence=confidence, ease=ease)
    session.add(new_idea)
    session.commit()
    return new_idea

def get_idea(idea_id: int) -> Idea:
    session = Session()
    idea = session.query(Idea).filter(Idea.id == idea_id).first()
    return idea

def update_idea(idea_id: int, title: str = None, notes: str = None, impact: int = None, confidence: int = None, ease: int = None) -> Idea:
    session = Session()
    idea = session.query(Idea).filter(Idea.id == idea_id).first()
    if idea:
        if title is not None:
            idea.title = title
        if notes is not None:
            idea.notes = notes
        if impact is not None:
            idea.impact = impact
        if confidence is not None:
            idea.confidence = confidence
        if ease is not None:
            idea.ease = ease
        session.commit()
    return idea

def delete_idea(idea_id: int) -> bool:
    session = Session()
    idea = session.query(Idea).filter(Idea.id == idea_id).first()
    if idea:
        session.delete(idea)
        session.commit()
        return True
    return False



In [5]:
# 1. Create (Add) a new idea
new_idea = add_idea(title="Innovative Idea", notes="A note on the innovative idea", impact=8, confidence=7, ease=9)
print(f"Added Idea: {new_idea.title}, ICE Score: {new_idea.ice_score}")




2023-12-19 16:43:11,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-19 16:43:11,016 INFO sqlalchemy.engine.Engine INSERT INTO ideas (title, notes, impact, confidence, ease) VALUES (?, ?, ?, ?, ?)
2023-12-19 16:43:11,016 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('Innovative Idea', 'A note on the innovative idea', 8, 7, 9)
2023-12-19 16:43:11,016 INFO sqlalchemy.engine.Engine COMMIT


DetachedInstanceError: Instance <Idea at 0x14a087d5190> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: https://sqlalche.me/e/14/bhk3)

In [ ]:
# Assuming the ID of the newly added idea is needed for further operations
new_idea_id = new_idea.id

# 2. Read (Retrieve) the idea by its ID
retrieved_idea = get_idea(new_idea_id)
if retrieved_idea:
    print(f"Retrieved Idea: {retrieved_idea.title}, ICE Score: {retrieved_idea.ice_score}")
else:
    print("Idea not found.")

# 3. Update the idea - let's change the title and impact
updated_idea = update_idea(new_idea_id, title="Updated Idea Title", impact=10)
if updated_idea:
    print(f"Updated Idea: {updated_idea.title}, New ICE Score: {updated_idea.ice_score}")
else:
    print("Idea not found or update failed.")

# 4. Delete the idea
if delete_idea(new_idea_id):
    print(f"Idea with ID {new_idea_id} deleted successfully.")
else:
    print(f"Failed to delete the idea with ID {new_idea_id}.")
